#### 1. Import thư viện

In [36]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import plotly.graph_objects as go

#### 2. Đọc dữ liệu và chuẩn hoá

In [37]:
df = pd.read_csv('Data/trending_videos_unique.csv')

In [38]:
df['published_at'] = pd.to_datetime(df['published_at'], errors='coerce')
df['snapshot_date'] = pd.to_datetime(df['snapshot_date'], errors='coerce')
df['trending_time_days'] = df['trending_time'].str.replace(' days', '').astype(int)
df['duration'] = pd.to_timedelta(df['duration'])
df['duration_second'] = df['duration'].dt.total_seconds()
category_dummies = pd.get_dummies(df['category'], prefix='category')
df = pd.concat([df, category_dummies], axis=1)

#### 3. Xây dựng ma trận tương quan

In [39]:
correlation_matrix = df[['view_count', 'like_count',
                         'comment_count', 'trending_time_days']].corr()
correlation_matrix

,view_count,like_count,comment_count,trending_time_days
view_count,1.000000,0.879615,0.288361,0.267698
like_count,0.879615,1.000000,0.365976,0.204503
comment_count,0.288361,0.365976,1.000000,0.118778
trending_time_days,0.267698,0.204503,0.118778,1.000000


#### 4. Chọn những cột cần thiết và cần dự đoán

In [40]:
feature_columns = ['view_count', 'like_count', 'comment_count', 'duration_second'] + list(category_dummies.columns)
X = df[feature_columns]
y = df['trending_time_days']

#### 5. Chia dữ liệu thành tập huấn luyện và kiểm tra

In [41]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f'Số lượng mẫu trong tập huấn luyện: {X_train.shape[0]}')
print(f'Số lượng mẫu trong tập kiểm tra: {X_test.shape[0]}')

Số lượng mẫu trong tập huấn luyện: 1848
Số lượng mẫu trong tập kiểm tra: 463


#### 6. Xây dựng mô hình dự đoán

In [42]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [43]:
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f'\nĐánh Giá Mô Hình Linear Regression:')
print(f'Sai số Trung bình Tuyệt đối (MAE): {mae:.2f} ngày')
print(f'Sai số Trung bình Bình phương (MSE): {mse:.2f} ngày^2')
print(f'Căn Bậc Hai của MSE (RMSE): {rmse:.2f} ngày')
print(f'Hệ số Xác định (R² Score): {r2:.2f}')


Đánh Giá Mô Hình Linear Regression:
Sai số Trung bình Tuyệt đối (MAE): 3.46 ngày
Sai số Trung bình Bình phương (MSE): 17.93 ngày^2
Căn Bậc Hai của MSE (RMSE): 4.23 ngày
Hệ số Xác định (R² Score): 0.16


#### 7. Biểu đồ so sánh

In [44]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=y_test, y=y_pred, mode='markers', name='Dự Đoán'))
fig.add_trace(go.Scatter(x=y_test, y=y_test, mode='lines', name='Thực Tế'))
fig.update_layout(title='So sánh Thực Tế và Dự Đoán Thời Gian Trending',
                  xaxis_title='Thời Gian Trending Thực tế (ngày)',
                  yaxis_title='Thời Gian Trending Dự Đoán (ngày)')
fig.show()

#### 8. Ví dụ

In [45]:
def predict_trending_time(view_count, like_count, comment_count, duration, category):
    duration_seconds = pd.to_timedelta(duration).total_seconds() 
    category_encoded = pd.get_dummies([category], prefix='category')
    category_encoded = category_encoded.reindex(columns=category_dummies.columns, fill_value=0)
    
    new_data = pd.DataFrame([[
        view_count, like_count, comment_count, duration_seconds
    ] + category_encoded.iloc[0].tolist()],
        columns=feature_columns
    )
    
    predicted_trending_time = model.predict(new_data)[0]
    return predicted_trending_time

predicted_days = predict_trending_time(
    view_count=10000,
    like_count=5000,
    comment_count=100,
    duration='00:00:59',
    category='Technology'
)

print(f'Thời gian Trending Dự đoán: {predicted_days:.2f} ngày')

Thời gian Trending Dự đoán: 6.92 ngày
